In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers

d:\AnaConda\envs\Cap\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

from transformers import AutoImageProcessor, TFEfficientFormerModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet,VGG16,ResNet50,EfficientNetB0
from tensorflow.keras.regularizers import l2

from sklearn.metrics import confusion_matrix,classification_report

import matplotlib.pyplot as plt

In [3]:
from transformers import AutoImageProcessor, TFEfficientFormerModel

In [4]:
image_processor = AutoImageProcessor.from_pretrained("snap-research/efficientformer-l1-300")
model = TFEfficientFormerModel.from_pretrained("snap-research/efficientformer-l1-300")

Some layers from the model checkpoint at snap-research/efficientformer-l1-300 were not used when initializing TFEfficientFormerModel: ['classifier', 'distillation_classifier']
- This IS expected if you are initializing TFEfficientFormerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFEfficientFormerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFEfficientFormerModel were initialized from the model checkpoint at snap-research/efficientformer-l1-300.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFEfficientFormerModel for predictions without further training.


In [3]:
dir="../Data/Image_set2/Normal_images/train"

In [4]:
import os

In [5]:
len(os.listdir(dir+"/Real"))

15179

In [6]:
len(os.listdir(dir+"/Fake"))

15051

In [7]:
real_path=dir+"/Real"
fake_path=dir+"/Fake"

In [8]:
import PIL
from PIL import Image

In [22]:
path="C:/Users/Akshat/Desktop/Capstone/Data/sampled_Faces/Frames/frame_fake/0.jpg"

In [23]:
image=Image.open(path)

In [26]:
img=cv2.imread("C:/Users/Akshat/Desktop/Capstone/Data/sampled_Faces/Frames/frame_fake/0.jpg")
img = cv2.resize(img, (224, 224))
scaled_image = img.astype('float32') / 255.0
img = np.expand_dims(scaled_image, axis=0)
features = model.predict(img)

1/1 [==============================] - 0s 21ms/step


In [31]:
type(features)

numpy.ndarray

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_2 (Flatten)         (None, 62720)             0         
                                                                 
 dense_4 (Dense)             (None, 256)               16056576  
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 18,314,817
Trainable params: 16,468,993
Non-trainable params: 1,845,824
__________________________________

In [21]:
inputs = image_processor(image, return_tensors="tf")
outputs = model(**inputs)

In [22]:
features=outputs.last_hidden_state

In [23]:
features.shape

TensorShape([1, 49, 448])

In [9]:
global_avg_pooling_layer = tf.keras.layers.GlobalAveragePooling1D()

In [29]:
output_features=global_avg_pooling_layer(features)
output_features1=pd.DataFrame(output_features.numpy())

In [55]:
features_array = np.empty((0, 448), dtype=float)

In [56]:
o=output_features.numpy()

In [57]:
o.shape

(1, 448)

In [39]:
df=output_features1

In [11]:
features_array = np.empty((0, 448), dtype=float)

In [14]:
ctr=1
for image in os.listdir(real_path):
    img=Image.open(real_path+"/"+image)
    inputs = image_processor(img, return_tensors="tf")
    outputs = model(**inputs)
    features=outputs.last_hidden_state    #[1,49,448]
    reduced_features=global_avg_pooling_layer(features)    #[1,448] for storage purposes, remove this layer to get all the features and just flatten
    feat=reduced_features.numpy()
    features_array = np.concatenate((features_array,feat), axis = 0)
    # if(ctr%100==0):
    print(f"Image {ctr} done")
    ctr+=1
    del feat
    img.close()

Image 1 done
Image 2 done
Image 3 done
Image 4 done
Image 5 done
Image 6 done
Image 7 done
Image 8 done
Image 9 done
Image 10 done
Image 11 done
Image 12 done
Image 13 done
Image 14 done
Image 15 done
Image 16 done
Image 17 done
Image 18 done
Image 19 done
Image 20 done
Image 21 done
Image 22 done
Image 23 done
Image 24 done
Image 25 done
Image 26 done
Image 27 done
Image 28 done
Image 29 done
Image 30 done
Image 31 done
Image 32 done
Image 33 done
Image 34 done
Image 35 done
Image 36 done
Image 37 done
Image 38 done
Image 39 done
Image 40 done
Image 41 done
Image 42 done
Image 43 done
Image 44 done
Image 45 done
Image 46 done
Image 47 done
Image 48 done
Image 49 done
Image 50 done
Image 51 done
Image 52 done
Image 53 done
Image 54 done
Image 55 done
Image 56 done
Image 57 done
Image 58 done
Image 59 done
Image 60 done
Image 61 done
Image 62 done
Image 63 done
Image 64 done
Image 65 done
Image 66 done
Image 67 done
Image 68 done
Image 69 done
Image 70 done
Image 71 done
Image 72 done
I

In [15]:
for image in os.listdir(fake_path):
    img=Image.open(fake_path+"/"+image)
    inputs = image_processor(img, return_tensors="tf")
    outputs = model(**inputs)
    features=outputs.last_hidden_state    #[1,49,448]
    reduced_features=global_avg_pooling_layer(features)    #[1,448] for storage purposes, remove this layer to get all the features and just flatten
    feat=reduced_features.numpy()
    features_array = np.concatenate((features_array,feat), axis = 0)
    # if(ctr%100==0):
    print(f"Image {ctr} done")
    ctr+=1
    del feat
    img.close()

Image 15180 done
Image 15181 done
Image 15182 done
Image 15183 done
Image 15184 done
Image 15185 done
Image 15186 done
Image 15187 done
Image 15188 done
Image 15189 done
Image 15190 done
Image 15191 done
Image 15192 done
Image 15193 done
Image 15194 done
Image 15195 done
Image 15196 done
Image 15197 done
Image 15198 done
Image 15199 done
Image 15200 done
Image 15201 done
Image 15202 done
Image 15203 done
Image 15204 done
Image 15205 done
Image 15206 done
Image 15207 done
Image 15208 done
Image 15209 done
Image 15210 done
Image 15211 done
Image 15212 done
Image 15213 done
Image 15214 done
Image 15215 done
Image 15216 done
Image 15217 done
Image 15218 done
Image 15219 done
Image 15220 done
Image 15221 done
Image 15222 done
Image 15223 done
Image 15224 done
Image 15225 done
Image 15226 done
Image 15227 done
Image 15228 done
Image 15229 done
Image 15230 done
Image 15231 done
Image 15232 done
Image 15233 done
Image 15234 done
Image 15235 done
Image 15236 done
Image 15237 done
Image 15238 do

In [16]:
features_array.shape

(30230, 448)

In [18]:
df=pd.DataFrame(features_array)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,438,439,440,441,442,443,444,445,446,447
0,0.319587,0.740929,0.464584,0.725391,-0.012810,-1.149684,-0.038594,-0.327014,-0.154114,0.384575,...,0.584388,0.371236,-0.107152,0.009291,0.768575,-0.685553,0.363147,-0.134060,0.298729,0.410601
1,0.161553,0.282407,0.303938,0.341088,-0.214515,-0.484006,-0.392677,-0.436100,-0.238849,0.040627,...,0.793374,0.149390,-0.052575,-0.197187,0.180801,-0.777508,0.225731,0.133878,0.147619,0.236145
2,0.176226,0.332912,0.183711,0.239652,-0.339653,-0.626031,-0.346819,-0.286595,-0.079800,0.123180,...,0.599684,0.212713,0.157118,-0.148664,0.091372,-0.650060,0.241216,0.132933,0.100103,-0.045666
3,0.117250,0.246266,0.040510,0.296838,-0.354080,-0.398558,-0.260375,-0.149370,-0.117103,0.228677,...,0.603982,0.299755,0.162576,-0.082298,0.042257,-0.574919,0.355091,0.163088,-0.007151,-0.346583
4,0.187924,0.029395,0.205600,0.142896,-0.162614,-0.471617,-0.374545,-0.373469,-0.125815,0.051613,...,0.758769,0.222033,0.211545,-0.209460,0.072756,-0.709315,0.301771,0.280353,0.137477,-0.074121


In [28]:
a=[]
for i in range(1,30231):
    if i<=15179:
        a.append(0)
    else:
        a.append(1)


In [29]:
len(a)

30230

In [30]:
df.shape

(30230, 448)

In [31]:
df["Label"]=a

In [32]:
df

,0,1,2,3,4,5,6,7,8,9,...,439,440,441,442,443,444,445,446,447,Label
0,0.319587,0.740929,0.464584,0.725391,-0.012810,-1.149684,-0.038594,-0.327014,-0.154114,0.384575,...,0.371236,-0.107152,0.009291,0.768575,-0.685553,0.363147,-0.134060,0.298729,0.410601,0
1,0.161553,0.282407,0.303938,0.341088,-0.214515,-0.484006,-0.392677,-0.436100,-0.238849,0.040627,...,0.149390,-0.052575,-0.197187,0.180801,-0.777508,0.225731,0.133878,0.147619,0.236145,0
2,0.176226,0.332912,0.183711,0.239652,-0.339653,-0.626031,-0.346819,-0.286595,-0.079800,0.123180,...,0.212713,0.157118,-0.148664,0.091372,-0.650060,0.241216,0.132933,0.100103,-0.045666,0
3,0.117250,0.246266,0.040510,0.296838,-0.354080,-0.398558,-0.260375,-0.149370,-0.117103,0.228677,...,0.299755,0.162576,-0.082298,0.042257,-0.574919,0.355091,0.163088,-0.007151,-0.346583,0
4,0.187924,0.029395,0.205600,0.142896,-0.162614,-0.471617,-0.374545,-0.373469,-0.125815,0.051613,...,0.222033,0.211545,-0.209460,0.072756,-0.709315,0.301771,0.280353,0.137477,-0.074121,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30225,0.536518,0.128462,0.328262,0.036840,0.165759,-0.185087,-0.321766,-0.407616,0.104830,-0.053136,...,-0.049187,-0.092920,-0.213430,0.958852,-0.647554,0.251818,0.117747,0.177863,0.303257,1
30226,0.575990,0.288435,0.352368,0.144937,0.114544,-0.267761,-0.338894,-0.434196,0.068426,0.039032,...,-0.017123,-0.108671,-0.120238,0.871784,-0.635884,0.269222,0.093033,0.112793,0.256271,1
30227,0.689964,0.162393,0.347239,0.244051,0.074362,-0.453982,-0.364238,-0.519222,0.026835,0.108343,...,-0.093766,0.155187,-0.332491,0.823720,-0.600377,0.178883,0.130200,0.196606,0.195224,1
30228,0.597484,0.103329,0.210109,0.316241,0.204683,-0.248902,-0.232389,-0.471840,-0.019540,0.100048,...,-0.236611,-0.056581,-0.227770,1.094657,-0.689593,0.189443,0.119813,0.248495,0.182252,1


In [86]:
df.to_csv("../Data/Celeb-df_vit.csv")

In [11]:
from keras.models import load_model

In [13]:
model=load_model("../Model/mbn_v2.h5",compile=False)

In [19]:
import cv2

In [32]:
features_array = np.empty((0,1), dtype=float)

In [36]:
ctr=1
for image in os.listdir(real_path):
    img=cv2.imread(real_path+"/"+image)
    img = cv2.resize(img, (224, 224))
    scaled_image = img.astype('float32') / 255.0
    img = np.expand_dims(scaled_image, axis=0)
    features = model.predict(img)
    features_array = np.concatenate((features_array,features), axis = 0)
    # if(ctr%100==0):
    print(f"Image {ctr} done")
    ctr+=1
    del features

1/1 [==============================] - 0s 22ms/step
Image 1 done
1/1 [==============================] - 0s 20ms/step
Image 2 done
1/1 [==============================] - 0s 19ms/step
Image 3 done
1/1 [==============================] - 0s 18ms/step
Image 4 done
1/1 [==============================] - 0s 19ms/step
Image 5 done
1/1 [==============================] - 0s 19ms/step
Image 6 done
1/1 [==============================] - 0s 18ms/step
Image 7 done
1/1 [==============================] - 0s 19ms/step
Image 8 done
1/1 [==============================] - 0s 18ms/step
Image 9 done
1/1 [==============================] - 0s 18ms/step
Image 10 done
1/1 [==============================] - 0s 18ms/step
Image 11 done
1/1 [==============================] - 0s 18ms/step
Image 12 done
1/1 [==============================] - 0s 19ms/step
Image 13 done
1/1 [==============================] - 0s 19ms/step
Image 14 done
1/1 [==============================] - 0s 18ms/step
Image 15 done
1/1 [==============

In [40]:

for image in os.listdir(fake_path):
    img=cv2.imread(fake_path+"/"+image)
    img = cv2.resize(img, (224, 224))
    scaled_image = img.astype('float32') / 255.0
    img = np.expand_dims(scaled_image, axis=0)
    features = model.predict(img)
    features_array = np.concatenate((features_array,features), axis = 0)
    # if(ctr%100==0):
    print(f"Image {ctr} done")
    ctr+=1
    del features

1/1 [==============================] - 0s 23ms/step
Image 15342 done
1/1 [==============================] - 0s 21ms/step
Image 15343 done
1/1 [==============================] - 0s 20ms/step
Image 15344 done
1/1 [==============================] - 0s 21ms/step
Image 15345 done
1/1 [==============================] - 0s 21ms/step
Image 15346 done
1/1 [==============================] - 0s 20ms/step
Image 15347 done
1/1 [==============================] - 0s 21ms/step
Image 15348 done
1/1 [==============================] - 0s 22ms/step
Image 15349 done
1/1 [==============================] - 0s 22ms/step
Image 15350 done
1/1 [==============================] - 0s 21ms/step
Image 15351 done
1/1 [==============================] - 0s 21ms/step
Image 15352 done
1/1 [==============================] - 0s 21ms/step
Image 15353 done
1/1 [==============================] - 0s 22ms/step
Image 15354 done
1/1 [==============================] - 0s 21ms/step
Image 15355 done
1/1 [=============================

In [45]:
df1=pd.DataFrame(features_array)

df1.head()

len(df1)

30394

In [56]:
df1=df1.drop(df1.index[15180:15344],axis=0)

In [57]:
df1

,0
0,1.000000
1,1.000000
2,1.000000
3,1.000000
4,1.000000
...,...
30389,0.007545
30390,0.001430
30391,0.003984
30392,0.004489


In [ ]:
df1.to_csv("../Data/mbnv2_predlabels_celebdf.csv")